In [1]:
#随机更换代理，是反爬的trick之一，但是在实践过程中，并不work；
#更换header也是，但是从爬取的实践经验来看，time.sleep()的设置是非常重要的；
#sleep的时间设置长些，也就意味着访问url的间隔时间会长些，这样可以有效避免反爬；
import re
import requests
import urllib
from bs4 import BeautifulSoup
import random
import pandas as pd
import csv
from itertools import cycle
import time
import numpy as np
import random
import tqdm



def get_data(page, HEADER):
    basic_url = 'https://www.douban.com/group/638298/discussion?start=%d'%(page*25)
    #basic_url = 'https://www.douban.com/group/638298/discussion?start=0'
    response = requests.get(basic_url, headers = HEADER, timeout= 60)
    html = response.text
    soup = BeautifulSoup(html, 'lxml')
    
    #获取帖子中的title,name,response,time;
    data = soup.find_all('tr')
    
    #filtered_data[i] = ['精品', 'title','name','response_nums','time'] or ['title','name','response_nums','time']
    filtered_data = [[i for i in i.text.strip().split('\n')if i.strip() != ''] for i in data[1:]]
    
    #获取帖子中的topic's url及发帖人id；
    topic_people_urls = [url.get('href') for url in soup.find_all('a')]
    topic_url = []
    people_url = []
    for topic_people_url in topic_people_urls:
        topic_people_url_ = topic_people_url.split('/')
        #topic_url in one page;
        if len(topic_people_url_) < 3:
            continue
        if topic_people_url_[-3] == 'topic':
            topic_url.append(topic_people_url)
        #people_url in one page;
        elif topic_people_url_[-3] == 'people':
            people_url.append(topic_people_url)
    
             
    return filtered_data, topic_url, people_url

def get_most_popular_cont(soup):
    dataOfMostPopular = soup.find_all('ul', "topic-reply popular-bd")[0]

    #最赞回应的发帖方式（电脑orApp）；
    waysToPostsInMostPopular = []
    for i in dataOfMostPopular.find_all('li'):
        try:
            wayToPost = i.find_all('span', 'via')[0].text
        except:
            wayToPost = ''
        waysToPostsInMostPopular.append(wayToPost)

    #最赞回应的发帖内容；
    contsOfMostPopular = [i.text for i in dataOfMostPopular.find_all('p')]

    #最赞回应用户的nickname；
    nickNamesOfMostPopular = [i.text.split('\n')[1] for i in dataOfMostPopular.find_all('h4')]

    #最赞回应用户的id；
    idsOfMostPopular = [i.find('a').get('href') for i in dataOfMostPopular.find_all('h4')]
    
    return waysToPostsInMostPopular, contsOfMostPopular, nickNamesOfMostPopular, idsOfMostPopular


def get_way_id_name_cont(soup):
    data = soup.find_all('div', 'reply-doc content')
    #发帖方式（电脑orApp）
    ways = []
    for i in data:
        try:
            way = i.find_all('span', 'via')[0].text
        except:
            way = ''
        ways.append(way)

    #回复内容；
    conts = [j.text for i in data for j in i.find_all('p')]

    #回复人的nickname;
    nickNames = [i.find_all('a')[0].text for i in data]
    
    #回复人的id
    ids = [i.find_all('a')[0].get('href') for i in data]
    
    return ways, conts, nickNames, ids


def get_content(topic_url, custom):
    
    allWaysOfReplyContents = []
    allContsOfReplyContents = []
    allNicknamesOfReplyContents = []
    allIdsOfReplyContents = []
    allWaysOfReplyContents_ = []
    allContsOfReplyContents_ = []
    allNicknamesOfReplyContents_ = []
    allIdsOfReplyContents_ = []
    imgNumsInTopic_ = []
    topicDocs_ = []
    topicWays_ = []
    most_popular = []
    
    for url in topic_url:
        HEADER = {'User-Agent': random.choice(HEADERS_LIST)}
        response = requests.get(url, headers=HEADER, timeout= 60)
        html = response.text
        soup = BeautifulSoup(html, 'lxml')
        
        #帖子中包含图片的数量；
        try:
            imgNumsInTopic = len(soup.find_all('div', "rich-content topic-richtext")[0].find_all('img'))
        except:
            imgNumsInTopic = 0
        imgNumsInTopic_.append(imgNumsInTopic)
        
        topic = [i for i in soup.find_all('div','topic-doc')[0].text.split('\n') if i != '']
        
        try:
            topicDocs = ''.join([i.text for i in soup.find_all('div', "rich-content topic-richtext")[0].find_all('p')])
        except:
            topicDocs = '出现list_index错误'
        topicWays = topic[-1]
        
        topicDocs_.append(topicDocs)
        topicWays_.append(topicWays)
        
        if custom == 'mostPopular':
            #最赞回应的发帖方式（电脑orApp）、内容、昵称及id；
            waysToPostsInMostPopular, contsOfMostPopular, nickNamesOfMostPopular, idsOfMostPopular = get_most_popular_cont(soup)
            allWaysOfReplyContents_.append(waysToPostsInMostPopular)
            allContsOfReplyContents_.append(contsOfMostPopular)
            allNicknamesOfReplyContents_.append(nickNamesOfMostPopular)
            allIdsOfReplyContents_.append(idsOfMostPopular)
        
        elif custom == '1stPage':
            #第一页的发帖方式（电脑orApp）、内容、昵称及id；
            waysOfReplyIn1stPage, contsOfReplyIn1stPage, nickNamesOfReplyIn1stPage, idsOfReplyIn1stPage = get_way_id_name_cont(soup)
            try:
                len_popular, _, _, _ = get_most_popular_cont(soup)
                index = len(len_popular)
            except:
                index = 'none'
            allWaysOfReplyContents_.append(waysOfReplyIn1stPage)
            allContsOfReplyContents_.append(contsOfReplyIn1stPage)
            allNicknamesOfReplyContents_.append(nickNamesOfReplyIn1stPage)
            allIdsOfReplyContents_.append(idsOfReplyIn1stPage)
            most_popular.append(index)
            
        elif custom == 'allPages':
            #全部页的发帖方式（电脑orApp）、内容、昵称及id；
            waysOfReplyIn1stPage, contsOfReplyIn1stPage, nickNamesOfReplyIn1stPage, idsOfReplyIn1stPage = get_way_id_name_cont(soup)
            allWaysOfReplyContents.extend(waysOfReplyIn1stPage)
            allContsOfReplyContents.extend(contsOfReplyIn1stPage)
            allNicknamesOfReplyContents.extend(nickNamesOfReplyIn1stPage)
            allIdsOfReplyContents.extend(idsOfReplyIn1stPage)
            
            for page in allPagesOfReplyContents:
                response = requests.get(page, headers = HEADER, timeout= 60)
                htmlOfPage = response.text
                soupOfPage = BeautifulSoup(html, 'lxml')
                ways, conts, nickNames, ids = get_way_id_name_cont(soupOfPage)
                allWaysOfReplyContents.extend(ways)
                allContsOfReplyContents.extend(conts)
                allNicknamesOfReplyContents.extend(nickNames)
                allIdsOfReplyContents.extend(ids)
                time.sleep(3)
            
            allWaysOfReplyContents_.append(allWaysOfReplyContents)
            allContsOfReplyContents_.append(allContsOfReplyContents)
            allNicknamesOfReplyContents_.append(allNicknamesOfReplyContents)
            allIdsOfReplyContents_.append(allIdsOfReplyContents)
            
        time.sleep(3)
            
    return imgNumsInTopic_, topicDocs_, topicWays_, allWaysOfReplyContents_, allContsOfReplyContents_, allNicknamesOfReplyContents_, allIdsOfReplyContents_, most_popular

In [36]:
HEADERS_LIST = [
            "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; AcooBrowser; .NET CLR 1.1.4322; .NET CLR 2.0.50727)",
            "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 6.0; Acoo Browser; SLCC1; .NET CLR 2.0.50727; Media Center PC 5.0; .NET CLR 3.0.04506)",
            "Mozilla/4.0 (compatible; MSIE 7.0; AOL 9.5; AOLBuild 4337.35; Windows NT 5.1; .NET CLR 1.1.4322; .NET CLR 2.0.50727)",
            "Mozilla/5.0 (Windows; U; MSIE 9.0; Windows NT 9.0; en-US)",
            "Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Win64; x64; Trident/5.0; .NET CLR 3.5.30729; .NET CLR 3.0.30729; .NET CLR 2.0.50727; Media Center PC 6.0)",
            "Mozilla/5.0 (compatible; MSIE 8.0; Windows NT 6.0; Trident/4.0; WOW64; Trident/4.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; .NET CLR 1.0.3705; .NET CLR 1.1.4322)",
            "Mozilla/4.0 (compatible; MSIE 7.0b; Windows NT 5.2; .NET CLR 1.1.4322; .NET CLR 2.0.50727; InfoPath.2; .NET CLR 3.0.04506.30)",
            "Mozilla/5.0 (Windows; U; Windows NT 5.1; zh-CN) AppleWebKit/523.15 (KHTML, like Gecko, Safari/419.3) Arora/0.3 (Change: 287 c9dfb30)",
            "Mozilla/5.0 (X11; U; Linux; en-US) AppleWebKit/527+ (KHTML, like Gecko, Safari/419.3) Arora/0.6",
            "Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.8.1.2pre) Gecko/20070215 K-Ninja/2.1.1",
            "Mozilla/5.0 (Windows; U; Windows NT 5.1; zh-CN; rv:1.9) Gecko/20080705 Firefox/3.0 Kapiko/3.0",
            "Mozilla/5.0 (X11; Linux i686; U;) Gecko/20070322 Kazehakase/0.4.5",
            "Mozilla/5.0 (X11; U; Linux i686; en-US; rv:1.9.0.8) Gecko Fedora/1.9.0.8-1.fc10 Kazehakase/0.5.6",
            "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.56 Safari/535.11",
            "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_7_3) AppleWebKit/535.20 (KHTML, like Gecko) Chrome/19.0.1036.7 Safari/535.20",
            "Opera/9.80 (Macintosh; Intel Mac OS X 10.6.8; U; fr) Presto/2.9.168 Version/11.52",
        ]


for page in tqdm(range(0, 8000)):
    
    #if count % 18 == 0:
     #   time.sleep(random.randint(220, 230))
    
    #if count % 37 == 0:
     #   time.sleep(random.randint(265, 280))
    
    #if count % 56 == 0:
     #   time.sleep(random.randint(303, 336))
    
    HEADER = {'User-Agent': random.choice(HEADERS_LIST)}
    
    s_time = time.time()
    
    filtered_data, topic_url, people_url = get_data(page, HEADER)
    #topic_url = [i for i in topic_url if i.split(':')[0] != 'https']
    for i, j in enumerate(topic_url):
        if j.split(':')[0] != 'https':
            topic_url[i] = topic_url[-1]
            
    time.sleep(5)
    
    imgNumsInTopic_, topicDocs_, topicWays_, allWaysOfReplyContents_, allContsOfReplyContents_, allNicknamesOfReplyContents_, allIdsOfReplyContents_, most_popular = get_content(topic_url, '1stPage')
    
    jingpin = []
    for i in filtered_data:
        try:
            jingpin.append(i[-5])
        except:
            jingpin.append('')
    
    try:
        titles = [i[-4] for i in filtered_data]
    except:
        titles = [i[0] for i in filtered_data]

    names = [i[-3] for i in filtered_data]
    responses_num = [i[-2] for i in filtered_data]
    times = [i[-1] for i in filtered_data]
    
    #if len(allWaysOfReplyContents_) != len(titles):
     #   allWaysOfReplyContents_.append(allWaysOfReplyContents_[-1])
    
    csv = pd.DataFrame({'title':titles, 'name':names, 'id':people_url,
                        'responses_num':responses_num, 'time':times, 'docs':topicDocs_,
                        'reply_content':allContsOfReplyContents_, 'img_nums':imgNumsInTopic_, 
                        'topic_url':topic_url, 'reply_id':allIdsOfReplyContents_, 'jinpin':jingpin, 
                        'popular_index':most_popular, 'topic_ways':topicWays_, 
                        'replyWays':allWaysOfReplyContents_})

    csv.to_csv(r'hhhhhhhh.csv', mode= 'a', header= False, encoding= 'utf_8_sig')
    time.sleep(random.randrange(6, 8, 1))
    e_time = time.time()
    log = '第%d页'%(page+1) + '||' + '发帖时间：%s'%times[-1] + '||' + '耗时：%f秒'%(e_time-s_time)
    print(log)

第3366页||发帖时间：2020-01-27||耗时：118.237561秒
第3367页||发帖时间：2020-01-27||耗时：116.147701秒
第3368页||发帖时间：2020-01-27||耗时：119.698167秒
第3369页||发帖时间：2020-01-27||耗时：119.947542秒
第3370页||发帖时间：2020-01-27||耗时：117.607666秒
第3371页||发帖时间：2020-01-27||耗时：121.654634秒
第3372页||发帖时间：2020-01-27||耗时：119.973910秒
第3373页||发帖时间：2020-01-27||耗时：120.057891秒
第3374页||发帖时间：2020-01-27||耗时：119.647747秒
第3375页||发帖时间：2020-01-26||耗时：118.158472秒
第3376页||发帖时间：2020-01-26||耗时：119.935842秒
第3377页||发帖时间：2020-01-26||耗时：121.745439秒
第3378页||发帖时间：2020-01-26||耗时：119.530033秒
第3379页||发帖时间：2020-01-26||耗时：119.495390秒
第3380页||发帖时间：2020-01-26||耗时：119.200465秒
第3381页||发帖时间：2020-01-26||耗时：117.931901秒
第3382页||发帖时间：2020-01-26||耗时：121.600883秒
第3383页||发帖时间：2020-01-26||耗时：120.837042秒
第3384页||发帖时间：2020-01-26||耗时：119.919998秒
第3385页||发帖时间：2020-01-26||耗时：118.872127秒
第3386页||发帖时间：2020-01-26||耗时：118.719752秒
第3387页||发帖时间：2020-01-26||耗时：118.119392秒
第3388页||发帖时间：2020-01-26||耗时：120.334309秒
第3389页||发帖时间：2020-01-26||耗时：119.635027秒
第3390页||发帖时间：2020-01-26||耗时：120.506153秒


第3571页||发帖时间：2020-01-15||耗时：120.242679秒
第3572页||发帖时间：2020-01-15||耗时：123.306212秒
第3573页||发帖时间：2020-01-15||耗时：120.852069秒
第3574页||发帖时间：2020-01-15||耗时：119.709854秒
第3575页||发帖时间：2020-01-15||耗时：122.382379秒
第3576页||发帖时间：2020-01-15||耗时：120.930193秒
第3577页||发帖时间：2020-01-15||耗时：120.150888秒
第3578页||发帖时间：2020-01-15||耗时：120.825714秒
第3579页||发帖时间：2020-01-15||耗时：118.471096秒
第3580页||发帖时间：2020-01-15||耗时：121.756196秒
第3581页||发帖时间：2020-01-15||耗时：117.425046秒
第3582页||发帖时间：2020-01-15||耗时：117.816572秒
第3583页||发帖时间：2020-01-15||耗时：119.212181秒
第3584页||发帖时间：2020-01-15||耗时：123.781627秒
第3585页||发帖时间：2020-01-14||耗时：118.452414秒
第3586页||发帖时间：2020-01-14||耗时：119.382097秒
第3587页||发帖时间：2020-01-14||耗时：118.839144秒
第3588页||发帖时间：2020-01-14||耗时：121.491524秒
第3589页||发帖时间：2020-01-14||耗时：119.445799秒
第3590页||发帖时间：2020-01-14||耗时：122.554255秒
第3591页||发帖时间：2020-01-14||耗时：123.231973秒
第3592页||发帖时间：2020-01-14||耗时：119.585379秒
第3593页||发帖时间：2020-01-14||耗时：119.525308秒
第3594页||发帖时间：2020-01-14||耗时：121.373009秒
第3595页||发帖时间：2020-01-14||耗时：121.696227秒


第3776页||发帖时间：2019-12-31||耗时：117.273891秒
第3777页||发帖时间：2019-12-31||耗时：117.648903秒
第3778页||发帖时间：2019-12-31||耗时：118.955568秒
第3779页||发帖时间：2019-12-31||耗时：117.039305秒
第3780页||发帖时间：2019-12-31||耗时：122.736656秒
第3781页||发帖时间：2019-12-31||耗时：120.199506秒
第3782页||发帖时间：2019-12-31||耗时：118.882102秒
第3783页||发帖时间：2019-12-31||耗时：117.722693秒
第3784页||发帖时间：2019-12-31||耗时：117.646727秒
第3785页||发帖时间：2019-12-31||耗时：119.344996秒
第3786页||发帖时间：2019-12-31||耗时：118.265882秒
第3787页||发帖时间：2019-12-30||耗时：118.788364秒
第3788页||发帖时间：2019-12-30||耗时：117.461810秒
第3789页||发帖时间：2019-12-30||耗时：117.610611秒
第3790页||发帖时间：2019-12-30||耗时：118.637947秒
第3791页||发帖时间：2019-12-30||耗时：118.796162秒
第3792页||发帖时间：2019-12-30||耗时：118.282472秒
第3793页||发帖时间：2019-12-30||耗时：118.338447秒
第3794页||发帖时间：2019-12-30||耗时：116.001197秒
第3795页||发帖时间：2019-12-30||耗时：118.325434秒
第3796页||发帖时间：2019-12-30||耗时：116.717994秒
第3797页||发帖时间：2019-12-30||耗时：118.606777秒
第3798页||发帖时间：2019-12-30||耗时：121.322482秒
第3799页||发帖时间：2019-12-30||耗时：121.128239秒
第3800页||发帖时间：2019-12-30||耗时：120.985638秒


KeyboardInterrupt: 